In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from functools import partial

In [2]:
parse_dates = ['마취시작시각', '마취종료시각','수술시작시각','수술종료시각','수술방입실시각','수술방퇴실시각','예상시간']


In [3]:
sur_rev = pd.read_csv("surgery reservation.csv",encoding = 'utf8')
sur = pd.read_csv("surgery.csv",encoding = 'utf8',parse_dates=parse_dates)

In [4]:
sur.head()

,,수술방id,수술일자,수술시작시각,수술종료시각,수술명,환자id,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,수술부서,예상시간,PID
0,1,62,20171201,2017-12-01 15:15:00,2017-12-01 16:10:00,Intrathecal infusion pump insertion,263847,2017-12-01 14:45:00,2017-12-01 16:12:00,2017-12-01 14:40:00,2017-12-01 16:15:00,2010246,마취통증의학과,01:00,263847
1,2,33,20171201,2017-12-01 08:10:00,2017-12-01 09:10:00,Repeat C/sec,50066324,2017-12-01 08:00:00,2017-12-01 09:15:00,2017-12-01 07:55:00,2017-12-01 09:15:00,2008219,산부인과,01:30,50066324
2,3,2,20171201,2017-12-01 13:00:00,2017-12-01 18:10:00,Revision septorhinoplasty,50341829,2017-12-01 12:45:00,2017-12-01 18:15:00,2017-12-01 12:42:00,2017-12-01 18:20:00,2009390,이비인후과,02:00,50341829
3,4,2,20171201,2017-12-01 14:45:00,2017-12-01 18:10:00,Corrective Rhinoplasty,50341829,2017-12-01 12:45:00,2017-12-01 18:15:00,2017-12-01 12:42:00,2017-12-01 18:20:00,2009179,성형외과,03:00,50341829
4,5,66,20171201,2017-12-01 13:00:00,2017-12-01 16:25:00,ORIF /c PFNA,60184681,2017-12-01 12:45:00,2017-12-01 16:30:00,2017-12-01 12:42:00,2017-12-01 16:30:00,2005050,정형외과,02:00,60184681


In [5]:
sur.shape

(22477, 15)

##### # 20181101~20181130 으로 선택 => 수술예약정보와 동일기간 맞추기 위하여

In [6]:
sur_range = sur[sur['수술일자'].between(20181101, 20181130, inclusive=True)].drop(columns=['   ','환자id'])

##### # 예상시간 => 과거 동일 수술 중 MAX 시간(20181101~20181130)

In [7]:
for i in range(len(sur_range)):
    if sur_range['예상시간'].iloc[i] == '  :  ':
        temp = []
        for j in range(len(sur)):
            if sur_range['수술명'].iloc[i] == sur['수술명'].iloc[j]:
                temp.append(sur['예상시간'].iloc[j])
        
        if len(temp) != 0:
            max_value = max(temp)
            sur_range['예상시간'].iloc[i] = max_value
            

C:\Users\LeeMH\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


##### # 과거 동일수술 없는경우 => 예상시간 = 수술종료시각 - 수술시작시각 (20181101~20181130)

In [8]:
for i in range(len(sur_range)):
    if sur_range['예상시간'].iloc[i] == '  :  ':
        temp = sur_range['수술종료시각'].iloc[i] - sur_range['수술시작시각'].iloc[i]
        aa = temp.total_seconds()
        hours = int(aa / 3600)
        minutes = int(aa / 60) % 60
        result = '0'+str(hours)+':'+str(minutes)
        #sur_range['예상시간'].iloc[i] = sur_range['수술종료시각'].iloc[i] - sur_range['수술시작시각'].iloc[i]
        sur_range['예상시간'].iloc[i] = result

##### # 예상시간 => 과거 동일 수술 중 MAX 시간 (all range)

In [9]:
for i in range(len(sur)):
    if sur['예상시간'].iloc[i] == '  :  ':
        temp = []
        for j in range(len(sur)):
            if sur['수술명'].iloc[i] == sur['수술명'].iloc[j]:
                temp.append(sur['예상시간'].iloc[j])
        
        if len(temp) != 0:
            max_value = max(temp)
            sur['예상시간'].iloc[i] = max_value
            

##### # 과거 동일수술 없는경우 => 예상시간 = 수술종료시각 - 수술시작시각 (all range)

In [10]:
sur = sur.drop(sur.index[6522]) # 수술시작,종료시간 없는 row

In [11]:
for i in range(len(sur)):
    if sur['예상시간'].iloc[i] == '  :  ':
        temp = sur['수술종료시각'].iloc[i] - sur['수술시작시각'].iloc[i]
        aa = temp.total_seconds()
        hours = int(aa / 3600)
        minutes = int(aa / 60) % 60
        result = '0'+str(hours)+':'+str(minutes)
        sur['예상시간'].iloc[i] = result

##### # process별 사이 시간

In [12]:
sur_range['수술방입실~마취시작'] = sur_range['마취시작시각'] - sur_range['수술방입실시각']
sur_range['마취시작~수술시작'] = sur_range['수술시작시각'] - sur_range['마취시작시각']
sur_range['수술시작~수술끝'] = sur_range['수술종료시각'] - sur_range['수술시작시각']
sur_range['수술끝~마취종료'] = sur_range['마취종료시각'] - sur_range['수술종료시각']
sur_range['마취종료~수술방퇴실'] = sur_range['수술방퇴실시각'] - sur_range['마취종료시각']
sur_range['수술총시간'] = sur_range['수술방퇴실시각'] - sur_range['수술방입실시각']

In [13]:
sur_range.head(20)

,수술방id,수술일자,수술시작시각,수술종료시각,수술명,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,수술부서,예상시간,PID,수술방입실~마취시작,마취시작~수술시작,수술시작~수술끝,수술끝~마취종료,마취종료~수술방퇴실,수술총시간
20531,9,20181101,2018-11-01 08:10:00,2018-11-01 08:40:00,"Correction of ingrowing nail on great toe, Lt....",2018-11-01 08:00:00,2018-11-01 08:45:00,2018-11-01 07:55:00,2018-11-01 08:45:00,2012269,성형외과,01:00,216276,00:05:00,00:10:00,00:30:00,00:05:00,0 days,00:50:00
20532,33,20181101,2018-11-01 14:55:00,2018-11-01 15:55:00,Flexible ureteroscopic nephrolithotripry,2018-11-01 14:45:00,2018-11-01 16:00:00,2018-11-01 14:40:00,2018-11-01 16:00:00,2008226,비뇨의학과,01:30,415653,00:05:00,00:10:00,01:00:00,00:05:00,0 days,01:20:00
20533,37,20181101,2018-11-01 08:30:00,2018-11-01 09:40:00,Impacted tooth extraction and window opening,2018-11-01 08:00:00,2018-11-01 09:45:00,2018-11-01 07:55:00,2018-11-01 09:45:00,2012651,장애인구강진료센터,02:30,10148208,00:05:00,00:30:00,01:10:00,00:05:00,0 days,01:50:00
20534,32,20181101,2018-11-01 12:50:00,2018-11-01 13:40:00,Laparoscopic Lt. ovarian cystectomy,2018-11-01 12:30:00,2018-11-01 13:45:00,2018-11-01 12:28:00,2018-11-01 13:45:00,2009209,산부인과,01:30,10148389,00:02:00,00:20:00,00:50:00,00:05:00,0 days,01:17:00
20535,11,20181101,2018-11-01 11:00:00,2018-11-01 12:55:00,Lt. Mastectomy + SN>AD,2018-11-01 10:45:00,2018-11-01 13:00:00,2018-11-01 10:40:00,2018-11-01 13:00:00,7930692,외과,02:00,20065054,00:05:00,00:15:00,01:55:00,00:05:00,0 days,02:20:00
20536,34,20181101,2018-11-01 09:00:00,2018-11-01 12:25:00,"Lt. Nephroureterectomy, laparoscopic",2018-11-01 08:00:00,2018-11-01 12:30:00,2018-11-01 07:55:00,2018-11-01 12:30:00,2010244,비뇨의학과,04:30,20069199,00:05:00,01:00:00,03:25:00,00:05:00,0 days,04:35:00
20537,33,20181101,2018-11-01 11:15:00,2018-11-01 12:50:00,TUR-BT,2018-11-01 11:00:00,2018-11-01 13:00:00,2018-11-01 10:55:00,2018-11-01 13:00:00,2010124,비뇨의학과,02:30,20129864,00:05:00,00:15:00,01:35:00,00:10:00,0 days,02:05:00
20538,36,20181101,2018-11-01 10:25:00,2018-11-01 11:40:00,OD)Evisceration /c HA implantation,2018-11-01 10:15:00,2018-11-01 11:45:00,2018-11-01 10:10:00,2018-11-01 11:45:00,2009098,안과,02:00,20222509,00:05:00,00:10:00,01:15:00,00:05:00,0 days,01:35:00
20539,33,20181101,2018-11-01 10:20:00,2018-11-01 10:35:00,Excision of urethral condyloma,2018-11-01 10:00:00,2018-11-01 10:45:00,2018-11-01 09:55:00,2018-11-01 10:45:00,2009392,비뇨의학과,01:00,20314271,00:05:00,00:20:00,00:15:00,00:10:00,0 days,00:50:00
20540,7,20181101,2018-11-01 08:40:00,2018-11-01 11:55:00,Sutotal gastrectomy,2018-11-01 08:00:00,2018-11-01 12:00:00,2018-11-01 07:55:00,2018-11-01 12:00:00,2009388,외과,03:00,20544591,00:05:00,00:40:00,03:15:00,00:05:00,0 days,04:05:00


##### # 집도의id list & 부서 list

In [14]:
doctor_list = []
department_list = []
for i in sur['집도의id']:
    if i not in doctor_list:
        doctor_list.append(i)
for i in sur['수술부서']:
    if i not in department_list:
        department_list.append(i)

##### # 의사별 수술방 사용 기록

In [15]:
doctor_room_dic = dict()

for i in doctor_list:
    doctor_room_dic.setdefault(i, [])
    for j in range(len(sur)):
        if i == sur['집도의id'].iloc[j]:
            doctor_room_dic[i].append(sur['수술방id'].iloc[j])
 

##### # 수술부서별 수술방 사용 기록

In [16]:
department_room_dic = dict()         
for i in department_list:
    department_room_dic.setdefault(i, [])
    for j in range(len(sur)):
        if i == sur['수술부서'].iloc[j]:
            department_room_dic[i].append(sur['수술방id'].iloc[j])  

##### # 협진수술

In [17]:
co_operation_df = pd.DataFrame()
duplicated_patient = sur_range[sur_range.duplicated(['PID'],keep=False)]
for i in range(len(duplicated_patient)):
    for j in range(i+1,len(duplicated_patient)):
        if duplicated_patient['PID'].iloc[i] == duplicated_patient['PID'].iloc[j]:
            if duplicated_patient['수술일자'].iloc[i] == duplicated_patient['수술일자'].iloc[j]:
                co_operation_df = pd.concat([co_operation_df,duplicated_patient.iloc[i:i+1]])
                co_operation_df = pd.concat([co_operation_df,duplicated_patient.iloc[j:j+1]])
                #print(i)
                
co_operation_df = co_operation_df.drop_duplicates(keep='first')

In [18]:
co_operation_df.head(15)

,수술방id,수술일자,수술시작시각,수술종료시각,수술명,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,수술부서,예상시간,PID,수술방입실~마취시작,마취시작~수술시작,수술시작~수술끝,수술끝~마취종료,마취종료~수술방퇴실,수술총시간
20555,1,20181101,2018-11-01 09:30:00,2018-11-01 10:25:00,Debridement and local flap on neck(operative s...,2018-11-01 08:00:00,2018-11-01 10:30:00,2018-11-01 07:55:00,2018-11-01 10:30:00,2009094,성형외과,01:30,140129029,00:05:00,01:30:00,00:55:00,00:05:00,0 days,02:35:00
20556,1,20181101,2018-11-01 08:15:00,2018-11-01 10:25:00,Thyroid lobectomy[R];CND[R],2018-11-01 08:00:00,2018-11-01 10:30:00,2018-11-01 07:55:00,2018-11-01 10:30:00,2006065,이비인후과,02:00,140129029,00:05:00,00:15:00,02:10:00,00:05:00,0 days,02:35:00
20626,9,20181102,2018-11-02 12:05:00,2018-11-02 12:25:00,Both T-tube insertion,2018-11-02 12:00:00,2018-11-02 20:00:00,2018-11-02 11:55:00,2018-11-02 20:00:00,2007111,이비인후과,00:30,170118385,00:05:00,00:05:00,00:20:00,07:35:00,0 days,08:05:00
20627,9,20181102,2018-11-02 13:15:00,2018-11-02 19:55:00,"Debridement of soft tissue defect on heel, Rt....",2018-11-02 12:00:00,2018-11-02 20:00:00,2018-11-02 11:55:00,2018-11-02 20:00:00,2012269,성형외과,04:00,170118385,00:05:00,01:15:00,06:40:00,00:05:00,0 days,08:05:00
20639,63,20181102,2018-11-02 09:20:00,2018-11-02 09:40:00,Ventriculo-peritoneal shunt,2018-11-02 08:00:00,2018-11-02 09:45:00,2018-11-02 07:50:00,2018-11-02 09:45:00,2008092,외상외과,00:30,180411140,00:10:00,01:20:00,00:20:00,00:05:00,0 days,01:55:00
20640,63,20181102,2018-11-02 08:45:00,2018-11-02 09:40:00,Ventriculo-peritoneal shunt,2018-11-02 08:00:00,2018-11-02 09:45:00,2018-11-02 07:55:00,2018-11-02 09:45:00,2009229,신경외과,02:00,180411140,00:05:00,00:45:00,00:55:00,00:05:00,0 days,01:50:00
20704,32,20181105,2018-11-05 14:45:00,2018-11-05 16:45:00,Segmental resection of small bowel,2018-11-05 13:30:00,2018-11-05 19:15:00,2018-11-05 13:29:00,2018-11-05 19:15:00,2009170,외과,01:00,160081121,00:01:00,01:15:00,02:00:00,02:30:00,0 days,05:46:00
20705,32,20181105,2018-11-05 16:45:00,2018-11-05 19:05:00,Diaphragm resection and primary repair,2018-11-05 13:30:00,2018-11-05 19:15:00,2018-11-05 13:29:00,2018-11-05 19:15:00,2007060,흉부외과,01:00,160081121,00:01:00,03:15:00,02:20:00,00:10:00,0 days,05:46:00
20706,32,20181105,2018-11-05 14:00:00,2018-11-05 19:10:00,2nd debulking OP,2018-11-05 13:30:00,2018-11-05 19:15:00,2018-11-05 13:29:00,2018-11-05 19:15:00,2004236,산부인과,02:30,160081121,00:01:00,00:30:00,05:10:00,00:05:00,0 days,05:46:00
20712,71,20181105,2018-11-05 10:25:00,2018-11-05 12:30:00,"metal removal, synotosis excision",2018-11-05 10:00:00,2018-11-05 15:00:00,2018-11-05 09:55:00,2018-11-05 15:00:00,2010495,정형외과,02:00,170197500,00:05:00,00:25:00,02:05:00,02:30:00,0 days,05:05:00


##### # 수술예약정보

In [19]:
sur_rev = sur_rev[sur_rev['확정']=='Y']

In [20]:
sur_rev_no_co_operation = sur_rev[sur_rev['협진수술여부']=='N']

In [21]:
sur_rev_co_operation = sur_rev[sur_rev['협진수술여부']=='Y']

##### # 수술정보에서 집도의id추출(변경수술방, PID, 수술일자)

In [22]:
doctor_id_dic = dict()
for i in range(len(sur_rev_no_co_operation)):
    for j in range(len(sur_range)):
        if sur_rev_no_co_operation['수술일자'].iloc[i] == sur_range['수술일자'].iloc[j]:
            if sur_rev_no_co_operation['PID'].iloc[i] == sur_range['PID'].iloc[j]:
                if sur_rev_no_co_operation['변경수술방'].iloc[i] == sur_range['수술방id'].iloc[j]:
                    doctor_id_dic[sur_rev_no_co_operation['집도의'].iloc[i]] = sur_range['집도의id'].iloc[j]
    
    #sur_rev_no_co_operation['집도의'].iloc[i] = 

In [23]:
doctor_id_dic['정대균'] = '2016214'
doctor_id_dic['박성진'] = '2008190'
doctor_id_dic['박성진'] = '2008190'
doctor_id_dic['의사(전문의)'] = '222'

In [24]:
temp = sur_rev.copy()

temp = temp[temp.집도의 != '박진섭']
temp = temp[temp.집도의 != '박민석']
temp = temp[temp.집도의 != '장재혁']
for i in range(len(temp)):
    temp['집도의'].iloc[i] = doctor_id_dic[temp['집도의'].iloc[i]]

In [25]:
sur_rev_id = temp.copy().drop(columns=['수술명','등록번호','신청수술방','신청수술방순서','변경수술방','변경수술방순서','확정','수정일시'])

In [26]:
sur_rev_id=sur_rev_id.rename(columns = {'집도의':'집도의id'})

In [27]:
sur_rev_id.head()

,,수술일자,협진수술여부,집도의id,요청사항,신청등록일시,PID
0,1,20181101,N,2012269,NaN,18/10/26 12:16:37.449870,216276
1,2,20181101,N,2008226,NaN,18/10/29 09:49:34.125604,415653
2,3,20181101,N,2012651,장애인치과치료입니다. 오전 첫방 부탁드립니다.\r\n,18/10/19 13:14:54.260538,10148208
3,4,20181101,N,2009209,교수님 오전 외래입니다. 오후 수술 부탁드립니다.,18/10/29 11:37:16.797422,10148389
4,5,20181101,N,7930692,NaN,18/10/25 15:00:07.810779,20065054


##### # 수술정보, 예약정보 병합

In [28]:
sur_merge = pd.merge(sur_range,sur_rev_id, on=['PID','수술일자','집도의id'],how='left')

In [29]:
sur_merge = sur_merge.drop(columns=['   '])

In [30]:
sur_merge.head()

,수술방id,수술일자,수술시작시각,수술종료시각,수술명,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,...,PID,수술방입실~마취시작,마취시작~수술시작,수술시작~수술끝,수술끝~마취종료,마취종료~수술방퇴실,수술총시간,협진수술여부,요청사항,신청등록일시
0,9,20181101,2018-11-01 08:10:00,2018-11-01 08:40:00,"Correction of ingrowing nail on great toe, Lt....",2018-11-01 08:00:00,2018-11-01 08:45:00,2018-11-01 07:55:00,2018-11-01 08:45:00,2012269,...,216276,00:05:00,00:10:00,00:30:00,00:05:00,0 days,00:50:00,N,NaN,18/10/26 12:16:37.449870
1,33,20181101,2018-11-01 14:55:00,2018-11-01 15:55:00,Flexible ureteroscopic nephrolithotripry,2018-11-01 14:45:00,2018-11-01 16:00:00,2018-11-01 14:40:00,2018-11-01 16:00:00,2008226,...,415653,00:05:00,00:10:00,01:00:00,00:05:00,0 days,01:20:00,N,NaN,18/10/29 09:49:34.125604
2,37,20181101,2018-11-01 08:30:00,2018-11-01 09:40:00,Impacted tooth extraction and window opening,2018-11-01 08:00:00,2018-11-01 09:45:00,2018-11-01 07:55:00,2018-11-01 09:45:00,2012651,...,10148208,00:05:00,00:30:00,01:10:00,00:05:00,0 days,01:50:00,N,장애인치과치료입니다. 오전 첫방 부탁드립니다.\r\n,18/10/19 13:14:54.260538
3,32,20181101,2018-11-01 12:50:00,2018-11-01 13:40:00,Laparoscopic Lt. ovarian cystectomy,2018-11-01 12:30:00,2018-11-01 13:45:00,2018-11-01 12:28:00,2018-11-01 13:45:00,2009209,...,10148389,00:02:00,00:20:00,00:50:00,00:05:00,0 days,01:17:00,N,교수님 오전 외래입니다. 오후 수술 부탁드립니다.,18/10/29 11:37:16.797422
4,11,20181101,2018-11-01 11:00:00,2018-11-01 12:55:00,Lt. Mastectomy + SN>AD,2018-11-01 10:45:00,2018-11-01 13:00:00,2018-11-01 10:40:00,2018-11-01 13:00:00,7930692,...,20065054,00:05:00,00:15:00,01:55:00,00:05:00,0 days,02:20:00,N,NaN,18/10/25 15:00:07.810779


##### # timestamp format 통일

In [31]:
for i in range(len(sur_merge)):
    data = datetime.strptime(str(sur_merge.수술일자.iloc[i]),'%Y%m%d')
    result = data.strftime('%Y-%m-%d')
    sur_merge.수술일자.iloc[i] = result

In [32]:

to_datetime_fmt = partial(pd.to_datetime, format='%Y-%m-%d')
sur_merge['수술일자'] = sur_merge['수술일자'].apply(to_datetime_fmt)

In [33]:
for i in range(len(sur_merge)):
    if str(sur_merge.신청등록일시.iloc[i]) != 'nan':
        data = datetime.strptime(str('20'+sur_merge['신청등록일시'].iloc[i][:8]),'%Y/%m/%d')
        result = data.strftime('%Y-%m-%d')
        sur_merge.신청등록일시.iloc[i] = result

In [34]:
to_datetime_fmt = partial(pd.to_datetime, format='%Y-%m-%d')
sur_merge['신청등록일시'] = sur_merge['신청등록일시'].apply(to_datetime_fmt)

##### # 긴급수술 추출(수술정보만 있거나, 신청등록일시 == 수술일시) & time_stamp

In [35]:
a = sur_merge[sur_merge.신청등록일시.isnull()]

In [36]:
b = sur_merge[sur_merge.신청등록일시.isnull() == False]

In [37]:
c = b[b['신청등록일시'] == b['수술일자']]

In [38]:
sur_no_emergence = b[b['신청등록일시'] != b['수술일자']]

In [39]:
sur_emergence = pd.concat([a,c]).sort_index()

In [40]:
for i in range(len(sur_no_emergence)):
    if sur_range.예상시간.iloc[i] == '  :30':
        sur_range.예상시간.iloc[i] = '00:30'
    if sur_range.예상시간.iloc[i] == ' 2:00':
        sur_range.예상시간.iloc[i] = '02:00'
    if sur_range.예상시간.iloc[i] == ' 1:00':
        sur_range.예상시간.iloc[i] = '01:00'
    if sur_range.예상시간.iloc[i] == ' 1:  ':
        sur_range.예상시간.iloc[i] = '01:00'
    data = datetime.strptime(str(sur_range.예상시간.iloc[i]),'%H:%M')
    result = data.strftime('%H:%M')
    sur_range.예상시간.iloc[i] = result

##### # category별 평균, 표준편차(20181101~20181130)

##### ## 예상시간 timestamp format 통일

In [41]:
for i in range(len(sur_range)):
    if sur_range.예상시간.iloc[i] == '  :30':
        sur_range.예상시간.iloc[i] = '00:30'
    if sur_range.예상시간.iloc[i] == ' 2:00':
        sur_range.예상시간.iloc[i] = '02:00'
    if sur_range.예상시간.iloc[i] == ' 1:00':
        sur_range.예상시간.iloc[i] = '01:00'
    if sur_range.예상시간.iloc[i] == ' 1:  ':
        sur_range.예상시간.iloc[i] = '01:00'
    data = datetime.strptime(str(sur_range.예상시간.iloc[i]),'%H:%M')
    result = data.strftime('%H:%M')
    sur_range.예상시간.iloc[i] = result

In [42]:
sur_range['예상시간'].head()

20531    01:00
20532    01:30
20533    02:30
20534    01:30
20535    02:00
Name: 예상시간, dtype: object

In [40]:
#temp = sur_range.copy()

In [41]:
#temp['예상시간'] = pd.to_datetime(temp['예상시간'],format='%H:%M')

##### ## category 추가

In [43]:
sur_range_with_cat = pd.DataFrame()

In [44]:
time_range = [sur_range[(sur_range['예상시간'] > '00:00') & (sur_range['예상시간'] <= '00:45')],#cat1
             sur_range[(sur_range['예상시간'] > '00:45') & (sur_range['예상시간'] <= '01:15')],#cat2
             sur_range[(sur_range['예상시간'] > '01:15') & (sur_range['예상시간'] <= '01:45')],#cat3
             sur_range[(sur_range['예상시간'] > '01:45') & (sur_range['예상시간'] <= '02:15')],#cat4
             sur_range[(sur_range['예상시간'] > '02:15') & (sur_range['예상시간'] <= '02:45')],#cat5
             sur_range[(sur_range['예상시간'] > '02:45') & (sur_range['예상시간'] <= '03:15')],#cat6
             sur_range[(sur_range['예상시간'] > '03:15')]]                                   #cat7

In [45]:
sur_range_with_cat = pd.DataFrame()
k = 1
for i in time_range:  
    temp1 = i
    
    temp1['category'] = 'cat'+str(k)
    print('frequency of cat',str(k),':',len(temp1))
    sur_range_with_cat = pd.concat([sur_range_with_cat, temp1])    
    k = k+1
sur_range_with_cat = sur_range_with_cat.sort_index()

frequency of cat 1 : 213
frequency of cat 2 : 454
frequency of cat 3 : 375
frequency of cat 4 : 404
frequency of cat 5 : 116
frequency of cat 6 : 229
frequency of cat 7 : 155


C:\Users\LeeMH\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


##### # category별 평균, 표준편차(all range)

##### ## 예상시간 timestamp format 통일

In [46]:
sur = sur[sur.예상시간 != 'nan']

In [47]:
for i in range(len(sur)):
    if sur.예상시간.iloc[i] == '  :30':
        sur.예상시간.iloc[i] = '00:30'
    if sur.예상시간.iloc[i] == ' 2:00':
        sur.예상시간.iloc[i] = '02:00'
    if sur.예상시간.iloc[i] == ' 1:00':
        sur.예상시간.iloc[i] = '01:00'
    if sur.예상시간.iloc[i] == ' 1:  ':
        sur.예상시간.iloc[i] = '01:00'
    if sur.예상시간.iloc[i] == ' 1:30':
        sur.예상시간.iloc[i] = '01:30'
    if sur.예상시간.iloc[i] == '1 :  ':
        sur.예상시간.iloc[i] = '01:00'
    if sur.예상시간.iloc[i] == ' 3:00':
        sur.예상시간.iloc[i] = '03:00'
    if sur.예상시간.iloc[i] == ' 0:10':
        sur.예상시간.iloc[i] = '00:10'
    if sur.예상시간.iloc[i] == ' 0:30':
        sur.예상시간.iloc[i] = '00:30'
    if sur.예상시간.iloc[i] == ' 5:00':
        sur.예상시간.iloc[i] = '05:00'
    if sur.예상시간.iloc[i] == '0 :30':
        sur.예상시간.iloc[i] = '00:30'
    if sur.예상시간.iloc[i] == '01:0 ':
        sur.예상시간.iloc[i] = '01:00'
    if sur.예상시간.iloc[i] == '02:0:':
        sur.예상시간.iloc[i] = '02:00'
    if sur.예상시간.iloc[i] == '30:00':
        sur.예상시간.iloc[i] = '00:30'
    if sur.예상시간.iloc[i] == '012:30':
        sur.예상시간.iloc[i] = '12:30'
    if sur.예상시간.iloc[i] == '1 : 0':
        sur.예상시간.iloc[i] = '01:00'
    if sur.예상시간.iloc[i] == '03:0 ':
        sur.예상시간.iloc[i] = '03:00' 
        

    #if i %50 == 0:
    #    print(i)
    data = datetime.strptime(str(sur.예상시간.iloc[i]),'%H:%M')
    result = data.strftime('%H:%M')
    sur.예상시간.iloc[i] = result

In [44]:
#sur.iloc[18400:18450]

In [45]:
#sur.예상시간.iloc[18408]

##### ## category 추가

In [48]:
time_range_sur = [sur[(sur['예상시간'] > '00:00') & (sur['예상시간'] <= '00:45')],#cat1
             sur[(sur['예상시간'] > '00:45') & (sur['예상시간'] <= '01:15')],#cat2
             sur[(sur['예상시간'] > '01:15') & (sur['예상시간'] <= '01:45')],#cat3
             sur[(sur['예상시간'] > '01:45') & (sur['예상시간'] <= '02:15')],#cat4
             sur[(sur['예상시간'] > '02:15') & (sur['예상시간'] <= '02:45')],#cat5
             sur[(sur['예상시간'] > '02:45') & (sur['예상시간'] <= '03:15')],#cat6
             sur[(sur['예상시간'] > '03:15')]]   

In [49]:
sur_with_cat = pd.DataFrame()
k = 1
for i in time_range_sur:  
    temp1 = i
    
    temp1['category'] = 'cat'+str(k)
    print('frequency of cat',str(k),':',len(temp1))
    sur_with_cat = pd.concat([sur_with_cat, temp1])    
    k = k+1
sur_with_cat = sur_with_cat.sort_index()

frequency of cat 1 : 2581
frequency of cat 2 : 5063
frequency of cat 3 : 4386
frequency of cat 4 : 4537
frequency of cat 5 : 1201
frequency of cat 6 : 2710
frequency of cat 7 : 1995


C:\Users\LeeMH\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


##### # category별 평균,표준편차(20181101~20181130)

In [47]:
department_list = list(department_room_dic.keys())
category_list = ['cat1','cat2','cat3','cat4','cat5','cat6','cat7']

In [48]:
a = sur_range_with_cat[(sur_range_with_cat['수술부서']==department_list[0]) & (sur_range_with_cat['category']==category_list[0])]

In [49]:
b = pd.to_timedelta(a["수술총시간"]).mean()

In [50]:
pd.to_timedelta(a["수술총시간"]).std()

Timedelta('0 days 00:17:24.413711')

In [74]:
sur_range_with_cat[['수술총시간']] = sur_range_with_cat[['수술총시간']].apply(lambda x:x.astype('timedelta64[m]'))

In [86]:
sur_mean = sur_range_with_cat.groupby(['수술부서','category'])[['수술총시간']].mean()

In [85]:
std_sur=sur_range_with_cat.groupby(['수술부서','category'])[['수술총시간']].std()

In [88]:
sur_mean.to_excel("mean_one_m.xlsx", encoding='utf-8')

In [90]:
std_sur.to_excel("std_one_m.xlsx", encoding='utf-8')

##### # category별 평균,표준편차(all_range)

In [50]:
department_list = list(department_room_dic.keys())
category_list = ['cat1','cat2','cat3','cat4','cat5','cat6','cat7']

In [51]:
sur_with_cat['수술총시간'] = sur_with_cat['수술방퇴실시각'] - sur_with_cat['수술방입실시각']

In [52]:
sur_with_cat.shape

(22473, 17)

In [54]:
sur_with_cat[['수술총시간']] = sur_with_cat[['수술총시간']].apply(lambda x:x.astype('timedelta64[m]'))

In [55]:
sur_mean = sur_with_cat.groupby(['수술부서','category'])[['수술총시간']].mean()

In [57]:
sur_std = sur_with_cat.groupby(['수술부서','category'])[['수술총시간']].std()

In [65]:
sur_with_cat.drop('   ', axis=1, inplace=True)

In [68]:
sur_with_cat.groupby(['category']).count()

,수술방id,수술일자,수술시작시각,수술종료시각,수술명,환자id,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,수술부서,예상시간,PID,수술총시간
category,,,,,,,,,,,,,,,
cat1,2581,2581,2579,2579,2581,2581,2580,2579,2580,2579,2581,2581,2581,2581,2579
cat2,5063,5063,5062,5062,5060,5063,5062,5062,5062,5062,5063,5063,5063,5063,5062
cat3,4386,4386,4385,4385,4384,4386,4385,4385,4385,4385,4386,4386,4386,4386,4385
cat4,4537,4537,4535,4535,4535,4537,4536,4533,4536,4532,4537,4537,4537,4537,4532
cat5,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201,1201
cat6,2710,2710,2709,2709,2703,2710,2709,2709,2709,2708,2710,2710,2710,2710,2708
cat7,1995,1995,1995,1995,1993,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995


In [70]:
surgery_freq =  sur_with_cat.groupby(['수술부서', 'category'])[['PID']].count()

In [71]:
surgery_freq.to_excel("surgery_freq.xlsx", encoding='utf-8')

In [66]:
sur_with_cat.columns

Index(['수술방id', '수술일자', '수술시작시각', '수술종료시각', '수술명', '환자id', '마취시작시각', '마취종료시각',
       '수술방입실시각', '수술방퇴실시각', '집도의id', '수술부서', '예상시간', 'PID', 'category',
       '수술총시간'],
      dtype='object')

In [59]:
sur_std.to_excel("std_sur_all.xlsx", encoding='utf-8')

In [60]:
sur_mean.to_excel("mean_sur_all.xlsx", encoding='utf-8')

In [56]:
sur_mean

수술총시간
수술부서      category            
구강악안면외과   cat1       62.000000
          cat2       66.906103
          cat3       98.692308
          cat4      168.782609
          cat6      367.857143
          cat7      333.684211
마취통증의학과   cat1       54.680000
          cat2       71.129630
          cat3      115.000000
          cat4      128.636364
비뇨의학과     cat1       54.341772
          cat2       59.862000
          cat3       87.305556
          cat4      128.288973
          cat5      142.253731
          cat6      184.451128
          cat7      285.944056
산부인과      cat1       38.492823
          cat2       71.462366
          cat3       96.930804
          cat4      129.634343
          cat5      162.753138
          cat6      191.245283
          cat7      275.600000
성형외과      cat1       57.650350
          cat2       86.255639
          cat3      101.879004
          cat4      140.535211
          cat5      186.551402
          cat6      227.294479
...                        ...
이비인후과     cat6      201.572687
          cat7      354.921429
장애인구강진료센터 cat2       74.222222
          cat3       94.392857
          cat4      110.500000
          cat5      127.368421
          cat6      165.909091
          cat7      186.674419
재활의학과     cat2       42.000000
정신건강의학과   cat1       26.928571
정형외과      cat1       50.283654
          cat2       96.173333
          cat3      126.778603
          cat4      172.830084
          cat5      199.654639
          cat6      252.160338
          cat7      313.039604
피부과       cat1       34.641509
          cat2      235.000000
          cat6      905.000000
호흡기알레르기내과 cat2       57.000000
          cat3      125.000000
          cat4       65.000000
흉부외과      cat1       47.341837
          cat2       83.488323
          cat3       67.342048
          cat4      141.211470
          cat5      176.590909
          cat6      242.160000
          cat7      381.702247

[100 rows x 1 columns]

In [53]:
sur_with_cat.head()

,,수술방id,수술일자,수술시작시각,수술종료시각,수술명,환자id,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,수술부서,예상시간,PID,category,수술총시간
0,1,62,20171201,2017-12-01 15:15:00,2017-12-01 16:10:00,Intrathecal infusion pump insertion,263847,2017-12-01 14:45:00,2017-12-01 16:12:00,2017-12-01 14:40:00,2017-12-01 16:15:00,2010246,마취통증의학과,01:00,263847,cat2,01:35:00
1,2,33,20171201,2017-12-01 08:10:00,2017-12-01 09:10:00,Repeat C/sec,50066324,2017-12-01 08:00:00,2017-12-01 09:15:00,2017-12-01 07:55:00,2017-12-01 09:15:00,2008219,산부인과,01:30,50066324,cat3,01:20:00
2,3,2,20171201,2017-12-01 13:00:00,2017-12-01 18:10:00,Revision septorhinoplasty,50341829,2017-12-01 12:45:00,2017-12-01 18:15:00,2017-12-01 12:42:00,2017-12-01 18:20:00,2009390,이비인후과,02:00,50341829,cat4,05:38:00
3,4,2,20171201,2017-12-01 14:45:00,2017-12-01 18:10:00,Corrective Rhinoplasty,50341829,2017-12-01 12:45:00,2017-12-01 18:15:00,2017-12-01 12:42:00,2017-12-01 18:20:00,2009179,성형외과,03:00,50341829,cat6,05:38:00
4,5,66,20171201,2017-12-01 13:00:00,2017-12-01 16:25:00,ORIF /c PFNA,60184681,2017-12-01 12:45:00,2017-12-01 16:30:00,2017-12-01 12:42:00,2017-12-01 16:30:00,2005050,정형외과,02:00,60184681,cat4,03:48:00


In [54]:
for i in department_list:
    for j in category_list:
        temp = sur_with_cat[(sur_with_cat['수술부서']==i) & (sur_with_cat['category']==j)]
        #temp.head()
        print(i,' ',j)
        print('-- avg :' ,temp['수술총시간'].mean())
        print('-- std :',temp['수술총시간'].std())

마취통증의학과   cat1
-- avg : 0 days 00:54:40.800000
-- std : 0 days 00:29:25.818790
마취통증의학과   cat2
-- avg : 0 days 01:11:07.777777
-- std : 0 days 00:33:18.321622
마취통증의학과   cat3
-- avg : 0 days 01:55:00
-- std : 0 days 00:08:39.615242
마취통증의학과   cat4
-- avg : 0 days 02:08:38.181818
-- std : 0 days 00:55:53.165126
마취통증의학과   cat5
-- avg : NaT
-- std : NaT
마취통증의학과   cat6
-- avg : NaT
-- std : NaT
마취통증의학과   cat7
-- avg : NaT
-- std : NaT
산부인과   cat1
-- avg : 0 days 00:38:29.569377
-- std : 0 days 00:34:41.598938
산부인과   cat2
-- avg : 0 days 01:11:27.741935
-- std : 0 days 00:58:01.280947
산부인과   cat3
-- avg : 0 days 01:36:55.848214
-- std : 0 days 00:40:38.019090
산부인과   cat4
-- avg : 0 days 02:09:38.060606
-- std : 0 days 01:06:08.785631
산부인과   cat5
-- avg : 0 days 02:42:45.188284
-- std : 0 days 01:13:23.953898
산부인과   cat6
-- avg : 0 days 03:11:14.716981
-- std : 0 days 01:32:25.150550
산부인과   cat7
-- avg : 0 days 04:35:36
-- std : 0 days 02:04:12.280187
이비인후과   cat1
-- avg : 0 days 00:56:32.44444

In [55]:
sur_with_cat

,,수술방id,수술일자,수술시작시각,수술종료시각,수술명,환자id,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,수술부서,예상시간,PID,category,수술총시간
0,1,62,20171201,2017-12-01 15:15:00,2017-12-01 16:10:00,Intrathecal infusion pump insertion,263847,2017-12-01 14:45:00,2017-12-01 16:12:00,2017-12-01 14:40:00,2017-12-01 16:15:00,2010246,마취통증의학과,01:00,263847,cat2,01:35:00
1,2,33,20171201,2017-12-01 08:10:00,2017-12-01 09:10:00,Repeat C/sec,50066324,2017-12-01 08:00:00,2017-12-01 09:15:00,2017-12-01 07:55:00,2017-12-01 09:15:00,2008219,산부인과,01:30,50066324,cat3,01:20:00
2,3,2,20171201,2017-12-01 13:00:00,2017-12-01 18:10:00,Revision septorhinoplasty,50341829,2017-12-01 12:45:00,2017-12-01 18:15:00,2017-12-01 12:42:00,2017-12-01 18:20:00,2009390,이비인후과,02:00,50341829,cat4,05:38:00
3,4,2,20171201,2017-12-01 14:45:00,2017-12-01 18:10:00,Corrective Rhinoplasty,50341829,2017-12-01 12:45:00,2017-12-01 18:15:00,2017-12-01 12:42:00,2017-12-01 18:20:00,2009179,성형외과,03:00,50341829,cat6,05:38:00
4,5,66,20171201,2017-12-01 13:00:00,2017-12-01 16:25:00,ORIF /c PFNA,60184681,2017-12-01 12:45:00,2017-12-01 16:30:00,2017-12-01 12:42:00,2017-12-01 16:30:00,2005050,정형외과,02:00,60184681,cat4,03:48:00
5,6,72,20171201,2017-12-01 08:25:00,2017-12-01 09:25:00,"1. Correction of blow out fracture(floor), Rt....",60325495,2017-12-01 08:00:00,2017-12-01 09:30:00,2017-12-01 07:55:00,2017-12-01 09:30:00,2011079,성형외과,02:00,60325495,cat4,01:35:00
6,7,2,20171201,2017-12-01 08:20:00,2017-12-01 09:40:00,Removal of Lt. nasolabial cyst - alotomy incision,60494038,2017-12-01 08:00:00,2017-12-01 09:45:00,2017-12-01 07:55:00,2017-12-01 09:45:00,2009390,이비인후과,01:00,60494038,cat2,01:50:00
7,8,8,20171201,2017-12-01 08:15:00,2017-12-01 09:10:00,Rt. endoscopic T1 (UBM),70511215,2017-12-01 08:00:00,2017-12-01 09:15:00,2017-12-01 07:55:00,2017-12-01 09:15:00,2009391,이비인후과,01:30,70511215,cat3,01:20:00
8,9,31,20171201,2017-12-01 08:20:00,2017-12-01 08:55:00,Laparoscopic RSO,80719191,2017-12-01 08:00:00,2017-12-01 09:00:00,2017-12-01 07:55:00,2017-12-01 09:05:00,2004236,산부인과,01:30,80719191,cat3,01:10:00
9,10,2,20171201,2017-12-01 11:20:00,2017-12-01 12:40:00,Rt. maxillay sinus mass biopsy,90015597,2017-12-01 11:00:00,2017-12-01 12:45:00,2017-12-01 10:55:00,2017-12-01 12:45:00,2009390,이비인후과,01:00,90015597,cat2,01:50:00


In [56]:
time_range_no_emergence = [sur_no_emergence[(sur_no_emergence['예상시간'] > '00:00') & (sur_no_emergence['예상시간'] <= '00:45')],#cat1
             sur_no_emergence[(sur_no_emergence['예상시간'] > '00:45') & (sur_no_emergence['예상시간'] <= '01:15')],#cat2
             sur_no_emergence[(sur_no_emergence['예상시간'] > '01:15') & (sur_no_emergence['예상시간'] <= '01:45')],#cat3
             sur_no_emergence[(sur_no_emergence['예상시간'] > '01:45') & (sur_no_emergence['예상시간'] <= '02:15')],#cat4
             sur_no_emergence[(sur_no_emergence['예상시간'] > '02:15') & (sur_no_emergence['예상시간'] <= '02:45')],#cat5
             sur_no_emergence[(sur_no_emergence['예상시간'] > '02:45') & (sur_no_emergence['예상시간'] <= '03:15')],#cat6
             sur_no_emergence[(sur_no_emergence['예상시간'] > '03:15')]]                                   #cat7

In [57]:
sur_no_emergence_with_cat = pd.DataFrame()
k = 1
for i in time_range_no_emergence:  
    temp1 = i
    
    temp1['category'] = 'cat'+str(k)
    print('frequency of cat',str(k),':',len(temp1))
    sur_no_emergence_with_cat = pd.concat([sur_no_emergence_with_cat, temp1])    
    k = k+1
sur_no_emergence_with_cat = sur_no_emergence_with_cat.sort_index()

C:\Users\kr6742\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


frequency of cat 1 : 183
frequency of cat 2 : 404
frequency of cat 3 : 330
frequency of cat 4 : 364
frequency of cat 5 : 107
frequency of cat 6 : 195
frequency of cat 7 : 135


##### # 요청사항 추가 data

In [59]:
#sur_no_emergence_with_cat.to_csv("surgery no emergence.csv",encoding='ms949')

In [57]:
parse_dates = ['수술일자','마취시작시각', '마취종료시각','수술시작시각','수술종료시각','수술방입실시각','수술방퇴실시각','예상시간']

In [67]:
sur_all_data = pd.read_csv("surgery no emergence_copy.csv",encoding = 'ms949',parse_dates=parse_dates)

In [68]:
sur_all_data.columns

Index(['수술방id', '수술일자', '수술시작시각', '수술종료시각', '수술명', '마취시작시각', '마취종료시각',
       '수술방입실시각', '수술방퇴실시각', '집도의id', '수술부서', '예상시간', 'PID', '수술방입실~마취시작',
       '마취시작~수술시작', '수술시작~수술끝', '수술끝~마취종료', '마취종료~수술방퇴실', '수술총시간', '협진수술여부',
       '요청사항', '신청등록일시', 'category', '첫방', '오전수술', '오후수술', '시간특정', '이어서',
       '수술방 지정(NO.)', '양방'],
      dtype='object')

In [69]:
sur_all_data.shape

(1718, 30)

In [70]:
sur_all_data['예상시간'] = sur_all_data['예상시간'].astype('object')

In [71]:
sur_all_data['예상시간'] = sur_all_data['예상시간'].apply(lambda x: str(x).split(" ")[1])

In [72]:
sur_all_data['예상시간(분)'] = sur_all_data['예상시간'].apply(lambda x: int(str(x).split(":")[0])*60 + int(str(x).split(":")[1]))

In [73]:
sur_all_data.fillna('0', inplace=True)

In [74]:
sur_all_data['예상시간(분)'] 

0        60
1        90
2       150
3        90
4       120
5       270
6       150
7       120
8       180
9       180
10      210
11      240
12       90
13      180
14       90
15      210
16       90
17       30
18       90
19      120
20       60
21      240
22       90
23      120
24      180
25       60
26      240
27       60
28       90
29      120
       ... 
1688    120
1689    120
1690     60
1691     30
1692    300
1693     90
1694     30
1695     40
1696     60
1697    120
1698     60
1699    150
1700     60
1701     60
1702     90
1703    300
1704    180
1705    240
1706     30
1707    180
1708    120
1709    120
1710    300
1711     60
1712    180
1713    180
1714     90
1715     90
1716    120
1717    120
Name: 예상시간(분), Length: 1718, dtype: int64

In [77]:
sur_all_data['수술총시간'] = sur_all_data['수술총시간'].apply(lambda x: pd.to_timedelta(x))

In [78]:
sur_all_data[['수술총시간(분)']] = sur_all_data[['수술총시간']].apply(lambda x: x.astype('timedelta64[m]'))

In [92]:
np.where(sur_all_data[['협진수술여부']] == 'Y')

(array([  22,   23,   64,   84,   85,   96,   97,  103,  146,  147,  148,
         152,  153,  154,  174,  175,  242,  243,  262,  263,  278,  279,
         280,  281,  337,  338,  344,  356,  357,  374,  375,  388,  390,
         391,  393,  394,  395,  401,  463,  495,  500,  501,  516,  538,
         539,  581,  607,  608,  706,  707,  745,  746,  778,  779,  798,
         799,  806,  807,  808,  813,  814,  828,  829,  838,  839,  842,
         843,  854,  855,  857,  862,  863,  877,  878,  879,  892,  893,
         896,  897,  917,  918,  958,  959,  960,  976,  977, 1019, 1041,
        1047, 1048, 1054, 1100, 1101, 1142, 1143, 1145, 1146, 1170, 1171,
        1172, 1173, 1212, 1213, 1227, 1228, 1240, 1241, 1247, 1248, 1262,
        1263, 1292, 1293, 1312, 1313, 1317, 1318, 1323, 1324, 1325, 1348,
        1349, 1410, 1411, 1469, 1470, 1474, 1501, 1504, 1505, 1556, 1557,
        1572, 1597, 1598, 1601, 1602, 1616, 1617, 1635, 1636, 1683, 1689,
        1704, 1706, 1707], dtype=int64

In [95]:
sur_all_data.iloc[[1,2]]

,수술방id,수술일자,수술시작시각,수술종료시각,수술명,마취시작시각,마취종료시각,수술방입실시각,수술방퇴실시각,집도의id,...,category,첫방,오전수술,오후수술,시간특정,이어서,수술방 지정(NO.),양방,예상시간(분),수술총시간(분)
1,33,2018-11-01,2018-11-01 14:55:00,2018-11-01 15:55:00,Flexible ureteroscopic nephrolithotripry,2018-11-01 14:45:00,2018-11-01 16:00:00,2018-11-01 14:40:00,2018-11-01 16:00:00,2008226,...,cat3,0,0,0,0,0,0,0,90,80.0
2,37,2018-11-01,2018-11-01 08:30:00,2018-11-01 09:40:00,Impacted tooth extraction and window opening,2018-11-01 08:00:00,2018-11-01 09:45:00,2018-11-01 07:55:00,2018-11-01 09:45:00,2012651,...,cat5,1,0,0,0,0,0,0,150,110.0


In [ ]:
sur_all_data

In [212]:
sur_all_data.iloc[np.where(sur_all_data[['협진수술여부']] == 'Y')[0], [12,30]]

,PID,예상시간(분)
22,140129029,210
23,140129029,210
64,20013802,120
84,170118385,270
85,170118385,270
96,180411140,150
97,180411140,150
103,180456532,180
146,160081121,270
147,160081121,270


In [209]:
sur_all_data.query('PID==261768').loc[::,'예상시간(분)']

828    510
829    510
Name: 예상시간(분), dtype: int64

In [211]:
co_surgery = sur_all_data.iloc[np.where(sur_all_data[['협진수술여부']] == 'Y')[0]].groupby('PID')[['예상시간(분)']].sum()
co_sur_pid_list = co_surgery.index.tolist()
for p in co_sur_pid_list:
    sur_all_data.set_value(sur_all_data[sur_all_data.PID==p].index, '예상시간(분)', co_surgery.xs(p).values[0])

261768
297596
10285200
20013802
30514595
40437400
50221046
60440587
60511115
70509146
70699564
80238355
80685157
90034390
100202122
100644780
110543758
130117300
130515889
140129029
150241489
150356675
160081121
160405804
160562433
160616509
170118385
170125174
170197500
170483489
180033243
180135154
180174173
180207804
180214871
180237735
180241381
180248708
180285619
180303588
180324169
180391037
180407617
180411140
180419045
180434198
180436950
180443273
180454343
180455876
180456532
180458860
180474789
180479166
180480760
180483660
180490873
180498273
180499926
180504293
180505519
180509227
180513174
180525900
180527335
180530571
180538489
180539158
180548563
180550564
180552672
180561582
180562675
180563739
180564097
180564204
180568226
960047925
970119966
990104692


C:\Users\LeeMH\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [214]:
sur_all_data.drop_duplicates('PID', inplace=True)

In [222]:
sur_all_data['category_id'] = sur_all_data['수술부서']+"_"+ sur_all_data['category']

In [223]:
sur_all_data.to_excel('surgery_data2.xlsx', encoding='utf-8', index=False)

In [225]:
sur_stat = pd.read_excel("./mean_sur_all.xlsx")

In [229]:
sur_stat.ffill(inplace=True)

In [232]:
sur_stat['category_id'] = sur_stat['수술부서']+"_"+ sur_stat['category']

In [237]:
sur_stat.rename(columns = {"수술총시간": "수술시간표준편차"} ,inplace=True)

In [240]:
sur_stat['수술시간표준편차'] = sur_stat['수술시간표준편차'] .apply(lambda x:round(float(x),2))

In [241]:
sur_stat['수술시간표준편차']

0      62.00
1      66.91
2      98.69
3     168.78
4     367.86
5     333.68
6      54.68
7      71.13
8     115.00
9     128.64
10     54.34
11     59.86
12     87.31
13    128.29
14    142.25
15    184.45
16    285.94
17     38.49
18     71.46
19     96.93
20    129.63
21    162.75
22    191.25
23    275.60
24     57.65
25     86.26
26    101.88
27    140.54
28    186.55
29    227.29
       ...  
70    201.57
71    354.92
72     74.22
73     94.39
74    110.50
75    127.37
76    165.91
77    186.67
78     42.00
79     26.93
80     50.28
81     96.17
82    126.78
83    172.83
84    199.65
85    252.16
86    313.04
87     34.64
88    235.00
89    905.00
90     57.00
91    125.00
92     65.00
93     47.34
94     83.49
95     67.34
96    141.21
97    176.59
98    242.16
99    381.70
Name: 수술시간표준편차, Length: 100, dtype: float64

In [242]:
sur_stat.to_excel('sur_stat.xlsx', index=False)

### # 안쓰는 코드

In [46]:
sur_want = sur_no_emergence[sur_no_emergence.요청사항.isnull() == False]

In [128]:
sur_want.요청사항.values

array(['장애인치과치료입니다. 오전 첫방  부탁드립니다.\r\n', '교수님 오전 외래입니다. 오후 수술 부탁드립니다.',
       '오후 외래입니다. 꼭 첫방 부탁드립니다', ..., '#T;OD)phaco,vit,MP,ILMP ',
       '#T:OS)phaco,vit,MP,ILMP',
       '#T:OS)encircling, vitrectomy, MP, SO inj.'], dtype=object)

In [75]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Twitter
twitter = Twitter()
kkma = Kkma()
hannanum = Hannanum()
komoran = Komoran()

C:\Users\kr6742\AppData\Local\Continuum\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
word_list = ['오전','첫','방','오후','외래','양방','수술','시','이어서','출장','교수']

In [114]:
for k in range(len(sur_want)):
    a = komoran.pos(sur_want.요청사항.values[k])
    temp = []
    for i,j in a:
        if i in word_list:
            temp.append(i)
        if j == 'SN':
            temp.append(i)
    print(temp)

java.lang.NullPointerExceptionPyRaisable: java.lang.NullPointerException

In [127]:
a = komoran.pos(sur_want.요청사항.values[3])

temp = []
for i,j in a:
    if i in word_list:
        temp.append(i)
    if j == 'SN':
        temp.append(i)

In [120]:
temp

['교수', '오전', '외래', '오후', '수술']

In [121]:
sur_want.요청사항.values[0]

'장애인치과치료입니다. 오전 첫방  부탁드립니다.\r\n'

In [124]:
a = komoran.pos('장애인치과치료입니다. 오전 첫방  부탁드립니다.')

In [125]:
a

[('장애', 'NNG'),
 ('인치', 'NNP'),
 ('과', 'JC'),
 ('치료', 'NNP'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF'),
 ('오전', 'NNP'),
 ('첫', 'MM'),
 ('방', 'NNP'),
 ('부탁', 'NNG'),
 ('드리', 'VV'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF')]

In [105]:
word_list = ['오전','첫','방','오후','외래','양방','수술','시','이어서','출장','교수']

In [94]:
komoran.pos('오전첫방부탁드립니다. c-arm, scope봅니다. ',flatten=True)

[('오전', 'NNP'),
 ('첫', 'MM'),
 ('방', 'NNP'),
 ('부탁', 'NNG'),
 ('드리', 'VV'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF'),
 ('c', 'SL'),
 ('-', 'SS'),
 ('arm', 'SL'),
 (',', 'SP'),
 ('scope', 'SL'),
 ('보', 'VV'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF')]

In [56]:
df = a[pd.notnull(a['수술시작시각'])]

In [48]:
#df1 = df[df['확정']=='Y']

In [59]:
len(sur_rev1)

1950